In [ ]:
import astropy.io.fits as fits
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn import preprocessing
import funclib
import skimage
import skimage as sk

import funclib

#### UNET

In [ ]:
'''
Ok, so not wieghting (just using MSE) does even worse than using crossentropyloss
Things to try: 
    - Using a weight of 3 for both doesnt help much at all
    - Using a weight of 10 for both just reduces GR and IG accuracy, doesnt help BP and DM accuracy (especially DM, BP does do a bit better) so total acc pretty bad
    - Even more so for 20
    - Raising them both higher and and also making dm more than bp (dm_weight=60, bp_weight=40) -> WTF.Doing that makes the accuracy and bp and dm like zero... 
Lets try adding input layer that brings out bp, dm??
    - Seems to be sort of helpfull? Gets like 11% acc on bps.Wait weird - training with sqaured again goes back to similar acc as with on layer, e.g. like near zero bp, dm acc
        Ok current UNet5 is with squared only, but not super usefull   
    - Try even more contrast by adding sqaured and fifth power? ---> NO! GIVES TERRIBLE TOTAL ACCURACY AND NEAR ZERO BP, DM ACCURACY
    - What about just fith? Still same level of terrible
    - Same for 7th (saved as UNet6)
    - Weird idea - use diff between binary seg output and inputs as second layer?
        Like, train it on a two (or three if perhaps scaling more does get bp) value seg, then took the difference of 
        that seg and the truths and used that as an input layer to train another model? Wouldnt be using truth values to train or anything. 
        Would that be sort of similar to "unsupervised NN" idea, where it reconstructs inputs using predictions, uses diff between those
        and true inputs to train?
        Ok, to do this I would need to rerun UNet1 and save the training preds to get the binary NN segs for the training data, cuase I've only saved them
        for the val data as of yet. Wait no! Dont want to save the preds it gets on the training data while training, but the preds that the fully trained
        model would make on the training preds. Or wait.. but those would probabaly be overfit to. Perhaps really I need to just use the validation outputs from UNet1
        as the training data for this UNet (both the imgs and the deltaBinImgs). And then I cant use that as the val data, so maybe use the train data my val data?
        Unfortunately, I did split them asymetrically, so there are 953 train files and 397 val files. 
        Lets not so this - on talking to Kevin it seems like maybe sort of too contrived
    - Kevin suggested using histogram normalization (not histogram matching) as a layer to perhaps bring out dm... but on looking at the results of
        applying this (as per Kevin's algorithm) seems like it might not be that usefull
Lets try training on new data (real data, with segmentations from segment_array_v2)! And also perhaps with some wieghting
    - BUT V2 NPY IMAGES ARE LIKE 4X AS BIG AS OTHERS, SO TAKES A HELL OF A LONG TIME 
    - Could cut them down. Or just cutting down batch size would result in traiing on same num pixels 
    - BUT probably should cut them down, since cutting batch size would mean most batches are like mostly the same image (less diverse)
    - Wait, still takes a very long time..
    - Maybe cuase theres just so many images? Lets just train on a subset
    - Save as UNet7
Huh.. does the similar to before (maybe worse?) where it does good on GR vs IG but terrible on others 
Tried on just binary data too and it does definitely way worse than trained on first light does
    - Save as UNet8
Ok, Benoit suggests training to just do IG, GR, BP
That does terrible. I wonder if its my custom loss function?
Lets try going going back to normal cross entropy loss
    - Save as UNet9
That works great on the GR, IG. Maybe it is my loss function? Can I weight with the built in? Yes I can!
    - try that with bp_wieght = 10
    - Save as UNet10
Wow that works great! so it is the loss function. Perhaps even getting too many. lets try agian with all 4 classes and a bit lower weight?
    - save as UNet11 (use bp_weight = dm_weight = 7)
Wow that works great too! Maybe a little too much weighting on both dm and bp (more so on dm). lets try 4 for dm, 5 for bp?
    - save as UNet11
Now lets see if we can do the same thing but train on real data? 
    - wait that probably wont work though since didnt work well on binary 
    - save as UNet12
    - Yeah I mean it does ok given the crappy true labels
And training on simulation (MURaM) data?
    - save as UNet13
'''

In [ ]:
# Compute total validation accuracy
pix_correct, ig_correct, dm_correct, gr_correct, bp_correct = 0, 0, 0, 0, 0
tot_pix, tot_ig, tot_dm, tot_gr, tot_bp = 0, 0, 0, 0, 0
for idx in range(396):
    true = np.load(f'../UNet2_outputs/true_{idx}.npy')
    preds = np.squeeze(np.load(f'../UNet2_outputs/pred_{idx}.npy'))
    pix_correct += len(np.where(preds.flatten() == true.flatten())[0]) #(preds == true).sum()
    tot_pix += len(preds.flatten()) # torch.numel(preds)
    ig_correct += len(np.where((preds.flatten() == true.flatten()) & (true.flatten() == 0))[0])
    tot_ig += len(np.where(true.flatten() == 0)[0])
    dm_correct += len(np.where((preds.flatten() == true.flatten()) & (true.flatten() == 0.5))[0])
    tot_dm += len(np.where(true.flatten() == 0.5)[0])
    gr_correct += len(np.where((preds.flatten() == true.flatten()) & (true.flatten() == 1))[0])
    tot_gr += len(np.where(true.flatten() == 1)[0])
    bp_correct += len(np.where((preds.flatten() == true.flatten()) & (true.flatten() == 1.5))[0])
    tot_bp += len(np.where(true.flatten() == 1.5)[0])

print(f"Total validation accuracy: {pix_correct/tot_pix*100:.2f}%") 
print(f'\t{ig_correct/tot_ig*100:.2f}% of intergranule pixels correctly identified')
print(f'\t{dm_correct/tot_dm*100:.2f}% of dim middle pixels correctly identified')
print(f'\t{gr_correct/tot_gr*100:.2f}% of granule pixels correctly identified')
print(f'\t{bp_correct/tot_bp*100:.2f}% of bright point pixels correctly identified')

# Display a random sample of images
import cv2
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8), (ax9, ax10)) = plt.subplots(5, 2, figsize=(5, 15)); axs = [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9, ax10]
ax1.set_title('"True" Labels')
ax2.set_title('Predicted Labels')
for i in range(0, 9, 2):
    idx = np.random.randint(0, 396)
    im = np.load(f'../UNet2_outputs/x_{idx}.npy')[0] # first index to get image
    true = np.load(f'../UNet2_outputs/true_{idx}.npy')
    preds = np.squeeze(np.load(f'../UNet2_outputs/pred_{idx}.npy'))
    axs[i].imshow(true)
    axs[i+1].imshow(preds)

In [ ]:
UNet_name = 'UNet13'
n_classes = 4

# Compute total validation accuracy
output_dir = f'../{UNet_name}_outputs'
pct_correct, pct_ig_correct, pct_dm_correct, pct_gr_correct, pct_bp_correct = funclib.compute_validation_results(output_dir, n_classes)
print(f'UNet: {UNet_name}')
print(f"Total validation accuracy: {pct_correct:.2f}%") 
print(f'\t{pct_ig_correct}% of intergranule pixels correctly identified')
print(f'\t{pct_dm_correct}% of dim middle pixels correctly identified')
print(f'\t{pct_gr_correct}% of granule pixels correctly identified')
print(f'\t{pct_bp_correct}% of bright point pixels correctly identified')

# Display a random sample of images
import cv2
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8), (ax9, ax10)) = plt.subplots(5, 2, figsize=(5, 15)); axs = [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9, ax10]
ax1.set_title('"True" Labels')
ax2.set_title('Predicted Labels')
for i in range(0, 9, 2):
    idx = np.random.randint(0, 396)
    im = np.load(f'{output_dir}/x_{idx}.npy')[0] # first index to get image
    true = np.load(f'{output_dir}/true_{idx}.npy')
    preds = np.squeeze(np.load(f'{output_dir}/pred_{idx}.npy'))
    axs[i].imshow(true, vmin=0, vmax=1.5)
    axs[i+1].imshow(preds, vmin=0, vmax=1.5)

#### Traditional ML

Note 1: currently using mostly default tunning parameters - could do more exploration to pick best ones

Note 2: currently using validation set from same image as training and test sets.
        Ideally could use new DKIST (DKIST_gband_series) for validation. 

Note 3: Why are validation accuracies higher than test accuracies? And why are test accuracies so high?
        Have I screwed something up?
        
        - Probably is just legit learning the thresholding perfectly, its a very simple metric
        - If thats the case, should get ~100% if test against segmented without BPs and DMs (when trained 
          on no BPs and DMs) and very close when trained on full. 

In [ ]:
'''
Try out various combinations of training and testing sets
'''

# Get data (DKIST image) and labels (using our non-ML id method)
data_OG = fits.open("Data/DKIST_example.fits")[0].data.byteswap().newbyteorder() # Not sure why this is big-endian to start with..
labels_OG = np.squeeze(fits.open("Data/DKIST_solarseg_output.fits")[0].data) # "c:\\Users\\lea\\OneDrive\\DKISTSegmentation\\DKISTSegmentation\\example_outputs\\DKIST\\segmented_data_DKIST_example.fits"
labels_noBPDMs = np.squeeze(fits.open("Data/DKIST_solarseg_output_noBPDMs.fits" )[0].data) # all files I created with run_segment_algorithm have first hdu shape (1,x,x)
data_50_36 = fits.open('Data/DKIST_gband_series_183653/SEG_VBI_2022_06_03T18_50_36_353_00430500_I_BPDWB_L1.fits')[1].data.byteswap().newbyteorder() # Not sure why this is big-endian to start with..
labels_50_36 = np.squeeze(fits.open('Data/DKIST_gband_series_183653/SEG_VBI_2022_06_03T18_50_36_353_00430500_I_BPDWB_L1.fits')[0].data)
data_51_58 = fits.open('Data/DKIST_gband_series_183653/SEG_VBI_2022_06_03T18_51_58_613_00430500_I_BPDWB_L1.fits')[1].data.byteswap().newbyteorder() # Not sure why this is big-endian to start with..
labels_51_58 = np.squeeze(fits.open('Data/DKIST_gband_series_183653/SEG_VBI_2022_06_03T18_51_58_613_00430500_I_BPDWB_L1.fits')[0].data)

# Prepare half (really 1/4) data for training and testing (use second half for first validation)
tt_data_OG = data_OG[0:int(np.shape(data_OG)[0]/2), 0:int(np.shape(data_OG)[1]/2)]
tt_labels_OG = labels_OG[0:int(np.shape(labels_OG)[0]/2), 0:int(np.shape(labels_OG)[1]/2)]
X_tt_OG, Y_tt_OG = funclib.pre_proccess(tt_data_OG, tt_labels_OG)
X_train_OG, X_test_OG, Y_train_OG, Y_test_OG = train_test_split(X_tt_OG, Y_tt_OG, test_size=0.4, random_state=20)

# Create validation set Y and Y using same steps as above
val_data_OG = data_OG[int(np.shape(data_OG)[0]/2):-1, int(np.shape(data_OG)[1]/2):-1]
val_labels_OG = labels_OG[int(np.shape(labels_OG)[0]/2):-1, int(np.shape(labels_OG)[1]/2):-1] #labels[int(np.shape(labels)[0]/2):-1, int(np.shape(labels)[1]/2):-1]
X_val_OG, Y_val_OG = funclib.pre_proccess(val_data_OG, val_labels_OG)

# Create "secondary" train and test sets (with a new data file)
X_tt_51_58, Y_tt_51_58 = funclib.pre_proccess(data_51_58, labels_51_58)
X_train_51_58, X_test_51_58, Y_train_51_58, Y_test_51_58 = train_test_split(X_tt_51_58, Y_tt_51_58, test_size=0.4, random_state=20)

# Create "secondary" validation set (a different new data file)
X_val_50_36, Y_val_50_36 = funclib.pre_proccess(data_50_36, labels_50_36)

# Create combined training set - WILL THE BOUNDARY CREATE PROBLEMS??? BUT I GUESS TRAIN_TEST_SPLIT (WHICH HONESTLY I SHOULDNT DO) WILL REMOVE THAT ISSUE
dataset1 = data_OG[0:int(np.shape(data_OG)[0]/2), 0:int(np.shape(data_OG)[1]/4)]
dataset2 = data_51_58[100:np.shape(dataset1)[0]+100, 100:np.shape(dataset1)[1]+100] # shift by 100 to aviod constant boundary
data_comb = np.concatenate((dataset1, dataset2), axis=1)
labelset1 = labels_OG[0:int(np.shape(labels_OG)[0]/2), 0:int(np.shape(labels_OG)[1]/4)]
labelset2 = labels_51_58[100:np.shape(labelset1)[0]+100, 100:np.shape(labelset1)[1]+100] # shift by 100 to aviod constant boundary
labels_comb = np.concatenate((labelset1, labelset2), axis=1)
X_tt_comb, Y_tt_comb = funclib.pre_proccess(data_comb, labels_comb)
X_train_comb, X_test_comb, Y_train_comb, Y_test_comb = train_test_split(X_tt_comb, Y_tt_comb, test_size=0.4, random_state=20)

# 1) RandomForestClassifier ###############################################################

# Train on OG train set
print('RF Trained on OG')
model = RandomForestClassifier(n_estimators=10)
model.fit(X_train_OG, Y_train_OG)

# Predict on OG test set
pred_test_OG =  model.predict(X_test_OG)
print('     accuracy on OG test set: ', metrics.accuracy_score(Y_test_OG, pred_test_OG))

# Predict on OG validation set
pred_val_OG =  model.predict(X_val_OG)
print('     accuracy on OG validation set: ', metrics.accuracy_score(Y_val_OG, pred_val_OG))

# Predict on validation set 2 
pred_val_50_36 =  model.predict(X_val_50_36)
print('     accuracy on 50_36 validation set: ', metrics.accuracy_score(Y_val_50_36, pred_val_50_36))

# Predict on combined test set
pred_test_comb =  model.predict(X_test_comb)
print('     accuracy on combined test set: ', metrics.accuracy_score(Y_test_comb, pred_test_comb))

# Train on 51_58 set 
print('RF trained on 51_58')
model = RandomForestClassifier(n_estimators=10)
model.fit(X_train2, Y_train2)

# Predict on OG validation set
pred_val_OG =  model.predict(X_val_OG)
print('     accuracy on OG validation set: ', metrics.accuracy_score(Y_val_OG, pred_val_OG))

# Predict on validation set 2 
pred_val_50_36 =  model.predict(X_val_50_36)
print('     accuracy on 50_36 validation set: ', metrics.accuracy_score(Y_val_50_36, pred_val_50_36))

# Predict on combined test set
pred_test_comb =  model.predict(X_test_comb)
print('     accuracy on combined test set: ', metrics.accuracy_score(Y_test_comb, pred_test_comb))

# Train on combined set 
print('RF trained on combined')
model = RandomForestClassifier(n_estimators=10)
model.fit(X_train_comb, Y_train_comb)

# Predict on OG validation set
pred_val_OG =  model.predict(X_val_OG)
print('     accuracy on OG validation set: ', metrics.accuracy_score(Y_val_OG, pred_val_OG))

# Predict on validation set 2 
pred_val_50_36 =  model.predict(X_val_50_36)
print('     accuracy on 50_36 validation set: ', metrics.accuracy_score(Y_val_50_36, pred_val_50_36))

# Predict on combined test set
pred_test_comb =  model.predict(X_test_comb)
print('     accuracy on combined test set: ', metrics.accuracy_score(Y_test_comb, pred_test_comb))

# Plot
fig, axs = plt.subplots(1, 3, figsize=(10, 3))
plt.suptitle('Random Forest (trained and tested on OG)', color=tc)
axs[0].imshow(val_data)
axs[0].set_title('Validation Data', color=tc)
axs[1].imshow(Y_val.reshape(np.shape(val_data)[0], np.shape(val_data)[1]))
axs[1].set_title('Validation True Labels', color=tc)
axs[2].imshow(pred_val.reshape(np.shape(val_data)[0], np.shape(val_data)[1]))
axs[2].set_title('Validation Predicted Labels', color=tc)
for ax in axs:
    ax.tick_params(labelcolor=tc, color=tc)

# 2) KNeighborsClassifier ###############################################################

# Train on OG train set
print('KNN Trained on OG')
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train_OG, Y_train_OG)

# Predict on OG test set
pred_test_OG =  model.predict(X_test_OG)
print('     accuracy on OG test set: ', metrics.accuracy_score(Y_test_OG, pred_test_OG))

# Predict on OG validation set
pred_val_OG =  model.predict(X_val_OG)
print('     accuracy on OG validation set: ', metrics.accuracy_score(Y_val_OG, pred_val_OG))

# Predict on validation set 2 
pred_val_50_36 =  model.predict(X_val_50_36)
print('     accuracy on 50_36 validation set: ', metrics.accuracy_score(Y_val_50_36, pred_val_50_36))

# Predict on combined test set
pred_test_comb =  model.predict(X_test_comb)
print('     accuracy on combined test set: ', metrics.accuracy_score(Y_test_comb, pred_test_comb))

# Train on 51_58 set 
print('KNN trained on 51_58')
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train_51_58, Y_train_51_58)

# Predict on OG validation set
pred_val_OG =  model.predict(X_val_OG)
print('     accuracy on OG validation set: ', metrics.accuracy_score(Y_val_OG, pred_val_OG))

# Predict on validation set 2 
pred_val_50_36 =  model.predict(X_val_50_36)
print('     accuracy on 50_36 validation set: ', metrics.accuracy_score(Y_val_50_36, pred_val_50_36))

# Predict on combined test set
pred_test_comb =  model.predict(X_test_comb)
print('     accuracy on combined test set: ', metrics.accuracy_score(Y_test_comb, pred_test_comb))

# Train on combined set 
print('KNN trained on combined')
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train_comb, Y_train_comb)

# Predict on OG validation set
pred_val_OG =  model.predict(X_val_OG)
print('     accuracy on OG validation set: ', metrics.accuracy_score(Y_val_OG, pred_val_OG))

# Predict on validation set 2 
pred_val_50_36 =  model.predict(X_val_50_36)
print('     accuracy on 50_36 validation set: ', metrics.accuracy_score(Y_val_50_36, pred_val_50_36))

# Predict on combined test set
pred_test_comb =  model.predict(X_test_comb)
print('     accuracy on combined test set: ', metrics.accuracy_score(Y_test_comb, pred_test_comb))

# Plot
fig, axs = plt.subplots(1, 3, figsize=(10, 3))
plt.suptitle('KNN (trained and tested on OG)', color=tc)
axs[0].imshow(val_data)
axs[0].set_title('Validation Data', color=tc)
axs[1].imshow(Y_val.reshape(np.shape(val_data)[0], np.shape(val_data)[1]))
axs[1].set_title('Validation True Labels', color=tc)
axs[2].imshow(pred_val.reshape(np.shape(val_data)[0], np.shape(val_data)[1]))
axs[2].set_title('Validation Predicted Labels', color=tc)
for ax in axs:
    ax.tick_params(labelcolor=tc, color=tc)


In [ ]:
'''
Same as above but simplify - create one big combined set to take train/test/val data from
'''

# Get data (DKIST image) and labels 
data = fits.open("Data/DKIST_example.fits")[0].data.byteswap().newbyteorder()[0:1365, 0:1365]
labels = np.squeeze(fits.open("Data/DKIST_solarseg_output.fits")[0].data)[0:1365, 0:1365]
for file in os.listdir('Data/DKIST_gband_series_183653/'):
        if file.startswith('SEG_'):
            path = 'Data/DKIST_gband_series_183653/'+file
            data1 = fits.open(path)[1].data.byteswap().newbyteorder()
            data = np.concatenate((data, data1), axis=1)
            labels1 = np.squeeze(fits.open(path)[0].data)
            labels = np.concatenate((labels, labels1), axis=1)

# Prepare half for training and testing (split along 'short' axis to encompass all)
tt_data = data[0:int(np.shape(data)[0]/2), :]
tt_labels = labels[0:int(np.shape(data)[0]/2), :]
X_tt, Y_tt = funclib.pre_proccess(tt_data, tt_labels)
X_train, X_test, Y_train, Y_test = train_test_split(X_tt, Y_tt, test_size=0.4, random_state=20)

# Prepare other half for validation
tt_data = data[int(np.shape(data)[0]/2):-1, :]
tt_labels = labels[int(np.shape(data)[0]/2):-1, :]
X_val, Y_val = funclib.pre_proccess(tt_data, tt_labels)


# 1) RandomForestClassifier ###############################################################

# Train on train set
print('RF')
model = RandomForestClassifier(n_estimators=10)
model.fit(X_train, Y_train)

# Predict on test set
pred_test =  model.predict(X_test)
print('     accuracy on test set: ', metrics.accuracy_score(Y_test, pred_test))

# Predict on OG validation set
pred_val =  model.predict(X_val)
print('     accuracy on validation set: ', metrics.accuracy_score(Y_val, pred_val))

# # 2) KNeighborsClassifier ###############################################################

# # Train on train set
# print('KNN')
# model = KNeighborsClassifier(n_neighbors=5)
# model.fit(X_train, Y_train)

# # Predict on test set
# pred_test =  model.predict(X_test)
# print('     accuracy on test set: ', metrics.accuracy_score(Y_test, pred_test))

# # Predict on OG validation set
# pred_val =  model.predict(X_val)
# print('     accuracy on validation set: ', metrics.accuracy_score(Y_val, pred_val))


In [ ]:
# # Add values of different Gabor filters as features
# num = 1
# kernels = []
# for theta in range(2):
#     theta = theta/4 * np.pi
#     for sigma in [1,3]:
#         for lambd in np.arange(0, np.pi, np.pi/4):
#             for gamma in [0.05, 0,5]:
#                 kernel = cv2.getGaborKernel((5, 5), sigma, theta, lambd, gamma, 0, ktype=cv2.CV_32F)
#                 kernels.append(kernel)
#                 filtered_img = cv2.filter2D(data1, cv2.CV_8UC3, kernel).reshape(-1)
#                 df['Gabor'+str(num)] = filtered_img
#                 num =  num + 1
#
# Add Canny edge detection as another feature
# edges = cv2.Canny(data).reshape(-1)
# df['Canny_edges'] = edges

##### Multiclass